# A Power System Expansion Problem
This notebook is based on a problem posed in Chapter 16 of the AIMMS documentation:<br>
https://documentation.aimms.com/_downloads/AIMMS_modeling.pdf <br>
In this notebook we work with an example of stochastic programming, which refers to a class of problems where the input data entails uncertainty.

In [1]:
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

**Problem Description:**<br>
We need to expand the capacity of a power system. The capacity of the system is referred to the maximum power the system can generate. There are 3 different types of power plants in the system: coal, nuclear and hydropower. The operating and capital costs associated with each type of power plant are different. We need to determine the expansion capacity for each plant type in $GW$, in order to meet the demand. We assume that there are no limitations on the number, or capacity of power plants. Furthermore, we can also import electricity from external sources to meet the demand, which results in additional costs.

# A Deterministic Expansion Model
Since the demand in this problem is not known, it introduces uncertainty in the problem. A simple way to tackle uncertainty is to assume there are a finite number of scenarios for the demand. We can consider each demand scenario separately, and solve the problem for that specific scenario. In other words, we can create a set of deterministic problems, and solve them. The optimization problem for each scenario can be summarized as follows:<br>
$\;\;\;\;\;\;$**Minimize:**$\;\;\;\;\;\;$ total cost of energy generation (capital, operation, and import)<br>
$\;\;\;\;\;\;$**Subject to:**<br>
$\;\;\;\;\;\;$Allocated capacity for each plant can be up to current capacity plus new design capacity<br>
$\;\;\;\;\;\;$Utilized capacity plus iported capacity are equal to demand<br>

**Indices:**<br>
$\;\;\;\;\;\;$$p$$\;\;\;\;\;\;\;\;\;\;\;$plant type<br>
$\;\;\;\;\;\;$$k$$\;\;\;\;\;\;\;\;\;\;\;$demand category<br>
**Parameters:**<br>
$\;\;\;\;\;\;$$e_{p}$$\;\;\;\;\;\;\;\;\;$current capacity of plant $p [GW]$<br>
$\;\;\;\;\;\;$$cc_{p}$$\;\;\;\;\;\;\;\;$daily capital cost of plant $p [10^{3}/GW]$<br>
$\;\;\;\;\;\;$$oc_{p}$$\;\;\;\;\;\;\;\;$daily operating cost of plant $p [10^{3}/GW]$<br>
$\;\;\;\;\;\;$$ic_{k}$$\;\;\;\;\;\;\;\;\;$import cost for category $k [10^{3}/GW]$<br>
$\;\;\;\;\;\;$$d_{k}$$\;\;\;\;\;\;\;\;\;$instantaneous demand for category $k [GW]$<br>
$\;\;\;\;\;\;$$du_{k}$$\;\;\;\;\;\;\;\;$duration of demand for category $k [h]$<br>
$\;\;\;\;\;\;$$r_{k}$$\;\;\;\;\;\;\;\;\;\;$required electricity for category $k [GWh]$<br>
**Variables:**<br>
$\;\;\;\;\;\;$$x_{p}$$\;\;\;\;\;\;\;\;\;$new capacity to be added for plant $p [GW]$<br>
$\;\;\;\;\;\;$$y_{pk}$$\;\;\;\;\;\;\;\;$allocation of capacity to demand $[GW]$<br>
$\;\;\;\;\;\;$$z_{k}$$\;\;\;\;\;\;\;\;\;$import of electricity for category $k [GWh]$<br>

So the mathematical formulation will look as follows:<br>
**Minimize:**
$$
\sum_{p}^{} cc_{p} (e_{p}+x_{p}) + \sum_{k}^{} \left( ic_{k}z_{k} + du_{k} \sum_{p}^{} oc_{p} y_{pk} \right)
$$
**Subject to:**
$$
\sum_{k}^{} y_{pk} \le e_{p} + x_{p} \;\;\;\;\;\;\;\;\;\;\;\; \forall p
$$
$$
z_{k} + du_{k} \sum_{p}^{} y_{pk} = r_{k} \;\;\;\;\;\;\;\; \forall k
$$

$$
x_{p} \ge 0 \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; \forall p
$$

$$
y_{pk} \ge 0 \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; \forall (p,k)
$$

$$
y_{nuclear,peak} = 0
$$

$$
z_{k} \ge 0 \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; \forall k
$$

In [2]:
demand_df = pd.DataFrame()
demand_df["Demand [GW]"] = ["base load", "peak load"]
demand_df["Scenario 1"] = [8.25, 10.75]
demand_df["Scenario 2"] = [10.00, 12.00]
demand_df["Scenario 3"] = [7.5, 10.00]
demand_df["Scenario 4"] = [9.00, 10.50]
demand_df["Duration"] = [24, 6]
demand_df["Import Cost"] = [200, 200]

demand_df

,Demand [GW],Scenario 1,Scenario 2,Scenario 3,Scenario 4,Duration,Import Cost
0,base load,8.25,10.0,7.5,9.0,24,200
1,peak load,10.75,12.0,10.0,10.5,6,200


In [3]:
cost_df = pd.DataFrame()
cost_df["Plant Type $p$"] = ["coal", "hydro", "nuclear"]
cost_df["current capacity $e_{p} [GW]$"] = [1.75, 2.00, 0]
cost_df["capital cost $cc_{p}$"] = [200, 500, 300]
cost_df["operating cost $oc_{p}$"] = [30.0, 10.0, 20.0]

cost_df

,Plant Type $p$,current capacity $e_{p} [GW]$,capital cost $cc_{p}$,operating cost $oc_{p}$
0,coal,1.75,200,30.0
1,hydro,2.00,500,10.0
2,nuclear,0.00,300,20.0


In [4]:
model = ConcreteModel()

#Indices
model.p = RangeSet(len(cost_df))
model.k = RangeSet(2)

#Parameters
def current_capacity(model, p):
    return cost_df.loc[p-1,"current capacity $e_{p} [GW]$"]
model.e = Param(model.p, initialize=current_capacity, within=Reals)

def capital_cost(model, p):
    return cost_df.loc[p-1,"capital cost $cc_{p}$"]
model.cc = Param(model.p, initialize=capital_cost, within=Reals)

def operating_cost(model, p):
    return cost_df.loc[p-1,"operating cost $oc_{p}$"]
model.oc = Param(model.p, initialize=operating_cost, within=Reals)

def import_cost(model, k):
    return demand_df.loc[k-1,"Import Cost"]
model.ic = Param(model.k, initialize=import_cost, within=Reals)

def duration(model, k):
    return demand_df.loc[k-1,"Duration"]
model.du = Param(model.k, initialize=duration, within=Reals)

def demand(model, k):
    return demand_df.loc[k-1,"Scenario 1"]
model.d = Param(model.k, initialize=demand, within=Reals, mutable=True)

def required_electricity(model, k):
    if k<2:
        return model.du[k] * model.d[k]
    else:
        return model.du[k] * (model.d[k] - model.d[k-1])
model.r = Param(model.k, initialize=required_electricity, within=Reals, mutable=True)

#Variables
model.x = Var(model.p, within=NonNegativeReals)
model.y = Var(model.p, model.k, within=NonNegativeReals)
model.z = Var(model.k, within=NonNegativeReals)

#Constraints
def capacity_alloc(model, p):
    return sum(model.y[p,k] for k in model.k) <= model.e[p] + model.x[p]
model.c1 = Constraint(model.p, rule=capacity_alloc)

def supply_demand(model, k):
    return model.z[k] + model.du[k] * sum(model.y[p,k] for p in model.p) == model.r[k]
model.c2 = Constraint(model.k, rule=supply_demand)

def peak_nuclear(model):
    return model.y[3,2] == 0
model.c3 = Constraint(rule=peak_nuclear)

#Objective Function
def rule_OF(model):
    return sum(model.cc[p] * (model.e[p]+model.x[p]) for p in model.p) + sum(model.ic[k]*model.z[k] + \
                            (model.du[k] * sum(model.oc[p] * model.y[p,k] for p in model.p)) for k in model.k)
model.objective = Objective(rule=rule_OF, sense=minimize)

opt = SolverFactory('glpk')
results = opt.solve(model)

Solving the problem for scenario 1, we find the following results:

In [5]:
model.x.pprint()
print("Total cost [10^3 $] =", value(model.objective))

x : Size=3, Index=p
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  0.75 :  None : False : False : NonNegativeReals
      2 :     0 :  6.25 :  None : False : False : NonNegativeReals
      3 :     0 :  -0.0 :  None : False : False : NonNegativeReals
Total cost [10^3 $] = 7055.0


# Other Demand Scenarios
Now if we wanted to solve the problem for other demand scenarios, we will need to update the demand values and solve the optimization problem again. Based on the demand values for scenario 2, we can update the parameter $d$ as follows. However, doing do will not change the results of the optimization. Why would this be happening?

In [6]:
def update_scenario(model, scenario_number):
    for i in range(len(model.d)):
        model.d[i+1] = demand_df.loc[i,"Scenario " + str(scenario_number)]
    return

update_scenario(model, 4)

results = opt.solve(model)

model.x.pprint()
print("Total cost [10^3 $] =", value(model.objective))

x : Size=3, Index=p
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  0.75 :  None : False : False : NonNegativeReals
      2 :     0 :  6.25 :  None : False : False : NonNegativeReals
      3 :     0 :  -0.0 :  None : False : False : NonNegativeReals
Total cost [10^3 $] = 7055.0


# Note:
The reason why the above method did not work, is because paramter $d$ is neither in the objective function, nor is it a part of any of the constraints. It is implicit in paramter $r$ which in turn is included in one of the constraints. Therefore, we need to update paramter $r$ as well in order to update the optimization problem. This can be done as follows:

In [7]:
def update_scenario(model, scenario_number):
    for i in range(len(model.d)):
        model.d[i+1] = demand_df.loc[i,"Scenario " + str(scenario_number)]
        if i<1:
            model.r[i+1] = model.du[i+1] * model.d[i+1]
        else:
            model.r[i+1] = model.du[i+1] * (model.d[i+1] - model.d[i])
    return


update_scenario(model, 2)

results = opt.solve(model)

model.x.pprint()
print("Total cost [10^3 $] =", value(model.objective))

x : Size=3, Index=p
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  0.25 :  None : False : False : NonNegativeReals
      2 :     0 :   8.0 :  None : False : False : NonNegativeReals
      3 :     0 :  -0.0 :  None : False : False : NonNegativeReals
Total cost [10^3 $] = 8160.0


Now we can solve the problem for all 4 different demand scenarios and compare the results.

In [9]:
scenarios = ["Scenario 1", "Scenario 2", "Scenario 3","Scenario 4"]
scenario_results = pd.DataFrame(index=scenarios)

for i in range(4):
    update_scenario(model, i+1)
    results = opt.solve(model)
    scenario_results.loc[scenarios[i], "Total Coal Capacity [GW]"] = value(model.x[1]) + value(model.e[1])
    scenario_results.loc[scenarios[i], "Total hydro Capacity [GW]"] = value(model.x[2]) + value(model.e[2])
    scenario_results.loc[scenarios[i], "Total nuclear Capacity [GW]"] = value(model.x[3]) + value(model.e[3])
    scenario_results.loc[scenarios[i], "Total Cost [10^3 $]"] = value(model.objective)
scenario_results

,Total Coal Capacity [GW],Total hydro Capacity [GW],Total nuclear Capacity [GW],Total Cost [10^3 $]
Scenario 1,2.50,8.25,0.0,7055.0
Scenario 2,2.00,10.00,0.0,8160.0
Scenario 3,2.50,7.50,0.0,6500.0
Scenario 4,1.75,8.75,0.0,7275.0


# Stochastic Programming Approach
In the stochastic programming approach, we do not consider each demand scenario separately. Instead, we assign a probability to each demand scenario, and include all scenarios together in the optimization problem. In order to do so, we need to change the objective function in a way that takes into account the costs of all scenarios, weighted by their respective probability.<br>
So the mathematical formulation will look as follows:<br>
**Minimize:**
$$
\sum_{p}^{} cc_{p} (e_{p}+x_{p}) + \sum_{s}^{} pr_{s}v_{s}
$$
**Subject to:**
$$
\sum_{k}^{} y_{pks} \le e_{p} + x_{p} \;\;\;\;\;\;\;\;\;\;\;\; \forall (p,s)
$$
$$
z_{ks} + du_{ks} \sum_{p}^{} y_{pks} = r_{ks} \;\;\;\;\;\;\;\; \forall (k,s)
$$
$$
\sum_{k}^{} \left( ic_{k}z_{ks} + du_{ks} \sum_{p}^{} oc_{p} y_{pks} \right) = v_{s} \;\;\;\;\;\;\;\; \forall s
$$

$$
x_{p} \ge 0 \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; \forall p
$$

$$
y_{pks} \ge 0 \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; \forall (p,k,s)
$$

$$
y_{nuclear,peak,s} = 0
$$

$$
z_{ks} \ge 0 \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; \forall (k,s)
$$

$$
v_{s} \ge 0 \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; \forall s
$$

In [11]:
del model

model = ConcreteModel()

#Indices
model.p = RangeSet(len(cost_df))
model.k = RangeSet(2)
model.s = RangeSet(4)

#Parameters
def current_capacity(model, p):
    return cost_df.loc[p-1,"current capacity $e_{p} [GW]$"]
model.e = Param(model.p, initialize=current_capacity, within=Reals)

def capital_cost(model, p):
    return cost_df.loc[p-1,"capital cost $cc_{p}$"]
model.cc = Param(model.p, initialize=capital_cost, within=Reals)

def operating_cost(model, p):
    return cost_df.loc[p-1,"operating cost $oc_{p}$"]
model.oc = Param(model.p, initialize=operating_cost, within=Reals)

def import_cost(model, k):
    return demand_df.loc[k-1,"Import Cost"]
model.ic = Param(model.k, initialize=import_cost, within=Reals)

def duration(model, k, s):
    return demand_df.loc[k-1,"Duration"]
model.du = Param(model.k, model.s, initialize=duration, within=Reals)

def demand(model, k, s):
    scenario_name = "Scenario " + str(s)
    return demand_df.loc[k-1, scenario_name]
model.d = Param(model.k, model.s, initialize=demand, within=Reals)

def required_electricity(model, k, s):
    if k<2:
        return model.du[k,s] * model.d[k,s]
    else:
        return model.du[k,s] * (model.d[k,s] - model.d[k-1,s])
model.r = Param(model.k, model.s, initialize=required_electricity, within=Reals)

model.pr = Param(model.s, initialize=0.25, within=Reals)

#Variables
model.x = Var(model.p, within=NonNegativeReals)
model.y = Var(model.p, model.k, model.s, within=NonNegativeReals)
model.z = Var(model.k, model.s, within=NonNegativeReals)
model.v = Var(model.s, within=NonNegativeReals)

#Constraints
def capacity_alloc(model, p, s):
    return sum(model.y[p,k,s] for k in model.k) <= model.e[p] + model.x[p]
model.c1 = Constraint(model.p, model.s, rule=capacity_alloc)

def supply_demand(model, k, s):
    return model.z[k,s] + model.du[k,s] * sum(model.y[p,k,s] for p in model.p) == model.r[k,s]
model.c2 = Constraint(model.k, model.s, rule=supply_demand)

def total_scenario_cost(model, s):
    return sum(model.ic[k]*model.z[k,s] + model.du[k,s]*sum(model.oc[p]*model.y[p,k,s] for p in model.p) \
               for k in model.k) == model.v[s]
model.c3 = Constraint(model.s, rule=total_scenario_cost)

def peak_nuclear(model, s):
    return model.y[3,2,s] == 0
model.c4 = Constraint(model.s, rule=peak_nuclear)

#Objective Function
def rule_OF(model):
    return sum(model.cc[p] * (model.e[p]+model.x[p]) for p in model.p) + \
            sum(model.pr[s]*model.v[s]for s in model.s)
model.objective = Objective(rule=rule_OF, sense=minimize)

opt = SolverFactory('glpk')
results = opt.solve(model)

In [12]:
scenario_results.loc["Stochastic", "Total Coal Capacity [GW]"] = value(model.x[1]) + value(model.e[1])
scenario_results.loc["Stochastic", "Total hydro Capacity [GW]"] = value(model.x[2]) + value(model.e[2])
scenario_results.loc["Stochastic", "Total nuclear Capacity [GW]"] = value(model.x[3]) + value(model.e[3])
scenario_results.loc["Stochastic", "Total Cost [10^3 $]"] = value(model.objective)

scenario_results

,Total Coal Capacity [GW],Total hydro Capacity [GW],Total nuclear Capacity [GW],Total Cost [10^3 $]
Scenario 1,2.50,8.25,0.00,7055.0
Scenario 2,2.00,10.00,0.00,8160.0
Scenario 3,2.50,7.50,0.00,6500.0
Scenario 4,1.75,8.75,0.00,7275.0
Stochastic,3.00,8.25,0.75,7605.0
